First, download the countdata and fasta file from Steven's git hub. I downloaded the count data manually, and here I downloaded the FASTA with ``curl``

In [3]:
#curl source_url > relative_path
!curl https://raw.githubusercontent.com/sr320/eimd-sswd/master/data/Phel_transcriptome.fasta > Phel_transcriptome.fa 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 39.4M  100 39.4M    0     0  5073k      0  0:00:07  0:00:07 --:--:-- 7878k


Familiarize myself with the data using ``head``

In [ ]:
cd /Users/natalielowell/Desktop/Seastar-HW

In [5]:
# First 2 lines
!head -n 2 Phel_transcriptome.fa

>Phel_contig_1
CAAATATATGAACGGTTGATTGTCAACGATTAGTACATGTTTTCATTGTTCCCCACGCCCGCCCCCCCCCACTCAAACATTTAAAGTGTGAAATATTATTTATCCACAAATTTCCTTAAACCTGCAAACTTGTCTGCTGTCTCTTATTGGAAGTTATGAAAAAGAACAACGGGTTTTCTTTAAAGGGTCTGCGTGCGATTTTCAACCTTTTGAGTAATAGCAGTTATTTTGATAACCGATTTTTTTCAAAGCTCAACAGCTTTTTAAAATAAGGAATCCTATAATGGCCAAACGAATACTATAAAAATAAGGGTTCTCTTAATTGTATAAAACGTATAATTTTATCAATTTTGGGACCGTGTAATTTTTTAAAGACCACAAGAATGTTACATACAACAAATAGACGAAACTCGTAGCTTTGGAAACTACGTCATGGGCGTTTGGTCAAAAGCTGGAGAGAAAGAGAGGTGGGGTGCCAGACTTAAGTAGTCACGTGATCTGACCAACGCACATCGGAAGCTCGATCGGATGAAATCTTCTCTATCGTTCTTGCGTCTATACGTGCTACGAAGAGCTGACAGAAGTTTGGACTTGTTTACTTCTTGCACCTGTTGATGGAACGGCCACGGACCTTGTCGCACGCACACCTGGAGCCAGTGCTCGGATCGACGCAACGGATGTACTGTCTTCCCCTTCCGCGTTTCTCAAGTAGGTACTCAAAGTCGTCCGCGTCGAAGTTGGCCTCGGCGTCCCTCTTCTCCAGCTCCTCCATGTCCTCCTCTGTGTAGTACGGGGTGACGAGCACCACCAGGGCGGCCACAATGGCCAGTGCTAGAAGACACTTCGTATTCATTCTGCTGGTGGTTGGATGTGCGCAAACAAGACAGGAGAGACTTATTAGAATC


In [6]:
# First 2 lines
!head -n 2 Phel_countdata.txt

Feature ID	Treated_FHL - Total gene reads	Treated_PH - Total gene reads	Treated_L - Total gene reads	Control_FHL - Total gene reads	Control_DB - Total gene reads	Control_PH - Total gene reads
Phel_contig_1	168	37	8	89	28	38


Adjusted the diff exp genes file manually in text wrangler and with the following script to remove ``"`` and to take just seq name.

``import sys
myfile = open(sys.argv[1], "r")				
new_file = open("justnames.txt", "w") 
for line in myfile:						   
	linelist=line.strip().split(" ")		   
	new_file.write(linelist[0] + "\n")
myfile.close()
new_file.close()``

^ this script is also in my FISH546 repo called ``justnames.py``

<br><br><br>
### Merged transcriptome file with diff exp genes file in R, a python script, and in Text Wrangler. R code used can be found in my git repo FISH546 and also below

##### Merge full transcriptome w tmp.sig to get just names and seqs of significantly diff exp genes

##### Manually changed transcriptome fasta to tab delimited file in text wrangler by replacing /n with /t and then replacing > with /n
##### Manually removed quotations from names in text wrangler for dif exp gene file
##### Bring edited dif exp gene file back in


##### Read in names of diff exp genes file
``newdif <- read.csv("justnames.txt", header = T, sep = "\t")
head(newdif) # check
names(newdif)[1] <- "Name"``

##### Read in transcriptome file
``Phel_transcriptome_tab <- read.delim("~/Desktop/Seastar-HW/Phel_transcriptome_tab.txt", header=FALSE)
trans <- Phel_transcriptome_tab
head(trans)
names(trans)[1:2] <- c("Name", "Seq")``

##### Merge files by name
``merged <- merge(newdif, trans, by = "Name", all.y = FALSE)
head(merged)``

##### Check that merge worked
Get length of just names file, then see if matches length of merged file###
``nrow(newdif)
nrow(merged)`` 

``write.table(merged[,1:2], file = "signames_seqs.txt", sep = "\n", row.names = FALSE, col.names = FALSE)``

<br><br>Removed quotations in textwrangler and put in carrots by searching and replacing ``Phel_`` with ``>Phel_``

Made a database using this new fasta file:

In [ ]:
!makeblastdb -in signames_seqs.fasta -dbtype nucl -out classdb

In [9]:
!blastn -query Gadus_morhua.gadMor1.cdna.all.fa -db classdb -num_threads 3 -evalue 1e-20 -outfmt 6 -out classtest.out.txt -max_target_seqs 5

/bin/sh: blastn: command not found


In [10]:
!grep 'ENSGMOT00000012564.1' Gadus_morhua.gadMor1.cdna.all.fa

>ENSGMOT00000012564.1 cdna genescaffold:gadMor1:GeneScaffold_2026:187345:189423:-1 gene:ENSGMOG00000011450.1 gene_biotype:protein_coding transcript_biotype:protein_coding gene_symbol:TUBA1C description:tubulin alpha 1c [Source:HGNC Symbol;Acc:HGNC:20768]
